In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
from dateutil.relativedelta import relativedelta

historical_dividend_aristocrats = ['MMM', 'AOS', 'ADM', 'T', 'CAH', 'CINF', 'CTAS', 'KO', 'CL',
                                   'CVX', 'DOV', 'ECL', 'EW', 'ESS', 'FAST', 'FITB', 'BEN', 'GD', 'GPC',
                                   'HRL', 'ITW', 'JNJ', 'KR', 'LEG', 'LIN', 'LOW', 'MCD', 'MDT', 'NUE',
                                   'PEP', 'PG', 'SHW', 'SWK', 'SYY', 'TGT', 'WBA', 'WMT', 'WM']

def get_historical_data(symbols, start_date, end_date):
    all_data = []

    for symbol in symbols:
        try:
            stock = yf.Ticker(symbol)
            hist_data = stock.history(start=start_date, end=end_date, interval="1wk")  # Weekly interval

            if hist_data.empty:
                print(f"No historical data for {symbol}.")
                continue

            # Calculate metrics for each row of hist_data
            for date, row in hist_data.iterrows():
                dividends = stock.dividends[:date]
                current_price = row['Close'] if not hist_data.empty else None
                dividend_sum = dividends.sum() if not dividends.empty else None
                dividend_yield = (dividend_sum / current_price) if current_price and dividend_sum else None

                # Handle stock.info fields with proper default values
                payout_ratio = stock.info.get('payoutRatio', None)
                eps = stock.info.get('epsTrailingTwelveMonths', None)
                pe_ratio = stock.info.get('trailingPE', None)
                roe = stock.info.get('returnOnEquity', None)
                free_cash_flow = stock.info.get('freeCashflow', None)
                gross_margin = stock.info.get('grossMargins', None)
                operating_margin = stock.info.get('operatingMargins', None)
                net_profit_margin = stock.info.get('profitMargins', None)
                price_to_book_ratio = stock.info.get('priceToBook', None)
                price_to_sales_ratio = stock.info.get('priceToSalesTrailing12Months', None)
                ev_ebitda = stock.info.get('enterpriseToEbitda', None)
                revenue_growth_rate = stock.info.get('revenueGrowth', None) * 100 if stock.info.get('revenueGrowth') else None
                earnings_growth_rate = stock.info.get('earningsQuarterlyGrowth', None) * 100 if stock.info.get('earningsQuarterlyGrowth') else None
                asset_turnover_ratio = stock.info.get('assetTurnover', None)
                inventory_turnover_ratio = stock.info.get('inventoryTurnover', None)
                price_to_cash_flow_ratio = stock.info.get('priceToCashflow', None)
                market_cap = stock.info.get('marketCap', None)
                interest_expense = stock.info.get('interestExpense', None)
                earnings_before_interest_and_taxes = stock.info.get('earningsBeforeInterestAndTaxes', None)
                interest_coverage_ratio = (earnings_before_interest_and_taxes / interest_expense) if interest_expense else None
                current_ratio = stock.info.get('currentRatio', None)
                quick_ratio = stock.info.get('quickRatio', None)
                total_assets = stock.info.get('totalAssets', None)
                employee_count = stock.info.get('fullTimeEmployees', None)
                price_volatility = hist_data['Close'].std() if not hist_data.empty else None
                avg_volume = hist_data['Volume'].mean() if not hist_data.empty else None

                all_data.append({
                    'symbol': symbol,
                    'date': date,
                    'close_price': current_price,
                    'dividend_yield': dividend_yield,
                    'payout_ratio': payout_ratio,
                    'eps': eps,
                    'pe_ratio': pe_ratio,
                    'roe': roe,
                    'free_cash_flow': free_cash_flow,
                    'gross_margin': gross_margin,
                    'operating_margin': operating_margin,
                    'net_profit_margin': net_profit_margin,
                    'price_to_book_ratio': price_to_book_ratio,
                    'price_to_sales_ratio': price_to_sales_ratio,
                    'ev_ebitda': ev_ebitda,
                    'revenue_growth_rate': revenue_growth_rate,
                    'earnings_growth_rate': earnings_growth_rate,
                    'asset_turnover_ratio': asset_turnover_ratio,
                    'inventory_turnover_ratio': inventory_turnover_ratio,
                    'price_to_cash_flow_ratio': price_to_cash_flow_ratio,
                    'market_cap': market_cap,
                    'interest_coverage_ratio': interest_coverage_ratio,
                    'current_ratio': current_ratio,
                    'quick_ratio': quick_ratio,
                    'total_assets': total_assets,
                    'employee_count': employee_count,
                    'price_volatility': price_volatility,
                    'avg_volume': avg_volume
                })

        except Exception as e:
            print(f"Failed to get data for {symbol}: {e}")

    return pd.DataFrame(all_data)

# Define the date range
end_date = datetime(2023, 12, 31, tzinfo=pytz.utc)
start_date = end_date - relativedelta(years=20)

# Get the data
historical_data = get_historical_data(historical_dividend_aristocrats, start_date, end_date)
print(historical_data.head())


  symbol                      date  close_price  dividend_yield  payout_ratio  \
0    MMM 2003-12-29 00:00:00-05:00    38.136707             NaN        2.0272   
1    MMM 2004-01-05 00:00:00-05:00    37.692997             NaN        2.0272   
2    MMM 2004-01-12 00:00:00-05:00    39.101898             NaN        2.0272   
3    MMM 2004-01-19 00:00:00-05:00    37.491734             NaN        2.0272   
4    MMM 2004-01-26 00:00:00-05:00    36.178867             NaN        2.0272   

    eps   pe_ratio      roe  free_cash_flow  gross_margin  ...  \
0  None  52.175785  0.24314    1.080262e+10       0.44763  ...   
1  None  52.175785  0.24314    1.080262e+10       0.44763  ...   
2  None  52.175785  0.24314    1.080262e+10       0.44763  ...   
3  None  52.175785  0.24314    1.080262e+10       0.44763  ...   
4  None  52.175785  0.24314    1.080262e+10       0.44763  ...   

   inventory_turnover_ratio  price_to_cash_flow_ratio   market_cap  \
0                      None                   

In [2]:
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39672 entries, 0 to 39671
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype                           
---  ------                    --------------  -----                           
 0   symbol                    39672 non-null  object                          
 1   date                      39672 non-null  datetime64[ns, America/New_York]
 2   close_price               39672 non-null  float64                         
 3   dividend_yield            38200 non-null  float64                         
 4   payout_ratio              38628 non-null  float64                         
 5   eps                       0 non-null      object                          
 6   pe_ratio                  36540 non-null  float64                         
 7   roe                       36540 non-null  float64                         
 8   free_cash_flow            38628 non-null  float64                         
 9   gross_

In [3]:
historical_df_cleaned = historical_data.dropna(axis=1, how='all')

# Drop rows with any NaN values
historical_df_cleaned = historical_df_cleaned.dropna(axis=0, how='any')

# Check for missing values in each column (optional, if you need more information)
missing_values = historical_df_cleaned.isna().sum()
print(missing_values[missing_values > 0])

print(historical_df_cleaned.info())

Series([], dtype: int64)
<class 'pandas.core.frame.DataFrame'>
Index: 28920 entries, 1048 to 39671
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype                           
---  ------                --------------  -----                           
 0   symbol                28920 non-null  object                          
 1   date                  28920 non-null  datetime64[ns, America/New_York]
 2   close_price           28920 non-null  float64                         
 3   dividend_yield        28920 non-null  float64                         
 4   payout_ratio          28920 non-null  float64                         
 5   pe_ratio              28920 non-null  float64                         
 6   roe                   28920 non-null  float64                         
 7   free_cash_flow        28920 non-null  float64                         
 8   gross_margin          28920 non-null  float64                         
 9   operating_margin      28920

In [4]:
int_columns = historical_df_cleaned.select_dtypes(include=['int64']).columns
historical_df_cleaned[int_columns] = historical_df_cleaned[int_columns].astype('float64')

# Ensure datetime64 columns are correctly formatted
historical_df_cleaned['date'] = pd.to_datetime(historical_df_cleaned['date'], errors='coerce')

# Handle object columns
historical_df_cleaned['symbol'] = historical_df_cleaned['symbol'].astype(str)

# Verify data types
print(historical_df_cleaned.dtypes)
print(historical_df_cleaned.info())

symbol                                            object
date                    datetime64[ns, America/New_York]
close_price                                      float64
dividend_yield                                   float64
payout_ratio                                     float64
pe_ratio                                         float64
roe                                              float64
free_cash_flow                                   float64
gross_margin                                     float64
operating_margin                                 float64
net_profit_margin                                float64
price_to_book_ratio                              float64
price_to_sales_ratio                             float64
ev_ebitda                                        float64
revenue_growth_rate                              float64
earnings_growth_rate                             float64
market_cap                                       float64
current_ratio                  

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
colab_notebooks_path = '/content/drive/My Drive/Colab Notebooks/'
historical_file_path = colab_notebooks_path + 'historical_df_cleaned.csv'

In [7]:
historical_df_cleaned.to_csv(historical_file_path, index=False)